In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select


import time,os

In [2]:
import itertools
import random
import datetime
import os
from selenium.webdriver.edge.service import Service



os.environ["DISPLAY"] = ":0"


# HELPER FUNCTIONS

In [15]:
# Quits the browser
def quitBrowser():
    try:
        browser.quit()
    except:
        pass


# Returns a new browser
def newBrowser():

    browserProfile = webdriver.ChromeOptions()
    browserProfile.add_argument('--headless')
    browserProfile.add_argument('--no-sandbox')
    browserProfile.add_argument('--disable-dev-shm-usage')
    browserProfile.add_argument('--disable-gpu')
    browserProfile.add_argument('--remote-debugging-port=9222')
    browserProfile.add_argument('--single-process')
    browserProfile.add_experimental_option(
        'prefs', {'intl.accept_languages': 'en,en_US'})
    ser = Service()
    ser.path = r'D:\document\DeepRacer\chromedriver.exe'
    browser = webdriver.Chrome(options=browserProfile, service=ser)
    browser.set_page_load_timeout(1800)  # 根据页面实际需要加载时间设置


    # In case chromedriver_binary does not work, use this:
    # path = XXX
    # browser = webdriver.Chrome(path, options=browserProfile)

    # If an element is not found, browser will try again every 0.5s until 3 seconds
    browser.implicitly_wait(3)

    return browser


# Logs in to AWS using the credentials in AWS_credentials.txt
def awsLogin():

    with open("AWS_credentials.txt", 'r') as f:
        [aws_id, username, password] = f.read().splitlines()

    # Build AWS Console URL with aws_id
    aws_id = str(aws_id)
    url = "https://%s.signin.aws.amazon.com/console" % aws_id

    # Open browser with the starting URL
    browser.get(url)
    browser.refresh()
    time.sleep(3)

    usernameInput = browser.find_elements_by_css_selector('form input')[1]
    passwordInput = browser.find_elements_by_css_selector('form input')[2]

    usernameInput.send_keys(username)
    passwordInput.send_keys(password)
    passwordInput.send_keys(Keys.ENTER)
    time.sleep(2.5)

    print(
        f"Successfully logged in to AWS account number {aws_id} with username {username}")


# General function to create a new model (used by cloneModel() and newModel())
# Note: rfchanges is not yet implemented (make changes to parameters in the reward function)
def createModel(modelname, description, track, hyperparams, maxtime,
                car=None, rewardfunction=None, rfchanges=None, testmode=False):

    #### PAGE 1 ####

    # Fill in the model name
    time.sleep(0.5)
    #modelnameInput = browser.find_element_by_id('awsui-input-1')
    # awsui-input awsui-input-type-text
    modelnameInput = browser.find_element_by_css_selector(
        'input[class="awsui-input awsui-input-type-text"]')
    modelnameInput.clear()
    modelnameInput.send_keys(modelname)

    # Fill in the training job description
    time.sleep(0.5)
    descriptionInput = browser.find_element_by_css_selector(
        'textarea[class="awsui-textarea"]')
    descriptionInput.clear()
    descriptionInput.send_keys(description)

    # Select the track
    time.sleep(0.5)
    trackValue = "arn:aws:deepracer:us-east-1::track/%s_track" % track
    trackButton = browser.find_element_by_css_selector(
        'input[type="radio"][value="%s"]' % trackValue)
    trackButton.click()

    # Click next (check page 3, part 3 for more robust xpath approach)
    time.sleep(0.5)
    nextButton = browser.find_elements_by_css_selector(
        'button[type="submit"] span[awsui-button-region="text"]')[3]
    nextButton.click()

    #### PAGE 2 ####

    # Click time trial (race type)
    time.sleep(0.5)
    raceType = "TIME_TRIAL"  # add to function parameters if needed
    raceTypeButton = browser.find_element_by_css_selector(
        'input[type="radio"][value="%s"]' % raceType)
    raceTypeButton.click()

    # Only select car if car argument is passed, else skip this part
    if car != None:

        time.sleep(0.5)

        # Expand car list
        allCarsList = browser.find_element_by_css_selector(
            'span[class="awsui-select-trigger-textbox"]')
        allCarsList.click()

        # Select desired car
        # number of retries with 1 sec wait inbetween (expanding car list takes time)
        retry = 10
        while retry > 0:
            time.sleep(1)
            try:
                carButton = browser.find_element_by_css_selector(
                    'div[title="%s"]' % car)
            except:
                retry -= 1  # this is executed when there was an error
            else:
                retry = 0  # this is executed when there was no error
        carButton.click()

    # Click next (check page 3, part 3 for more robust xpath approach)
    time.sleep(0.5)
    nextButton2 = browser.find_elements_by_css_selector(
        'button[type="submit"] span[awsui-button-region="text"]')[1]
    nextButton2.click()

    #### PAGE 3, PART 1 REWARD FUNCTION ####

    # Only fill out reward function if argument is passed, else leave pre-filled reward function
    if rewardfunction != None:

        time.sleep(0.5)

        # Click into code editor for reward function
        codeEditor = browser.find_elements_by_css_selector(
            'span[class="ace_keyword"]')[0]
        actionChain1 = webdriver.ActionChains(browser)
        actionChain1.move_to_element(codeEditor)
        actionChain1.click()
        actionChain1.perform()

        # Select all code and delete
        actionChain2 = webdriver.ActionChains(browser)
        # use Keys.CONTROL for a Windows system
        actionChain2.key_down(Keys.COMMAND)
        actionChain2.send_keys('a')
        actionChain2.key_up(Keys.COMMAND)
        actionChain2.send_keys(Keys.DELETE)
        actionChain2.perform()

        # Insert reward function
        # Note: COMMAND+C / COMMAND+V does not work with Selenium as Chrome blocks it because of security reasons
        # NOT YET IMPLEMENTED: WOULD BE EASIER TO INSERT reward_function.txt to S3 with Boto3
        # reward_function_to_console(rewardfunction)

    #### PAGE 3, PART 2 HYPERPARAMETERS ####

    # Expand hyperparameter menu
    time.sleep(0.5)
    hyperparamsExpand = browser.find_element_by_css_selector(
        'awsui-expandable-section[class="algorithm-settings"]')
    hyperparamsExpand.click()

    # Select desired batch size
    time.sleep(0.5)
    batchsizeButton = browser.find_element_by_css_selector(
        'input[type="radio"][value="%i"]' % hyperparams["batchsize"])
    batchsizeButton.click()

    # Enter desired number of epochs
    time.sleep(0.5)
    epochsField = browser.find_element_by_css_selector(
        'input[name="request.TrainingConfig.Hyperparameters.num_epochs"]')
    epochsField.clear()
    epochsField.send_keys(hyperparams["epochs"])

    # Enter desired learning rate
    time.sleep(0.5)
    learningrateField = browser.find_element_by_css_selector(
        'input[name="request.TrainingConfig.Hyperparameters.lr"]')
    learningrateField.clear()
    learningrateField.send_keys(str(hyperparams["learningrate"]))

    # Enter desired entropy
    time.sleep(0.5)
    entropyField = browser.find_element_by_css_selector(
        'input[name="request.TrainingConfig.Hyperparameters.beta_entropy"]')
    entropyField.clear()
    entropyField.send_keys(str(hyperparams["entropy"]))

    # Enter desired discount factor
    time.sleep(0.5)
    discountField = browser.find_element_by_css_selector(
        'input[name="request.TrainingConfig.Hyperparameters.discount_factor"]')
    discountField.clear()
    discountField.send_keys(str(hyperparams["discount"]))

    # Enter desired episodes between updates
    time.sleep(0.5)
    episodesUpdateField = browser.find_element_by_css_selector(
        'input[name="request.TrainingConfig.Hyperparameters.num_episodes_between_training"]')
    episodesUpdateField.clear()
    episodesUpdateField.send_keys(str(hyperparams["episodesUpdate"]))

    #### PAGE 3, PART 3 STOP CONDITION ####

    # Enter desired maximum training time in minutes
    time.sleep(0.5)
    episodesUpdateField = browser.find_element_by_css_selector(
        'input[name="request.TrainingConfig.TerminationConditions.MaxTimeInMinutes"]')
    episodesUpdateField.clear()
    episodesUpdateField.send_keys(str(maxtime))

    # Press "Create model", but only if testmode is False
    if testmode == False:
        time.sleep(0.5)
        createModelButton = browser.find_element_by_xpath(
            '//button[@type="submit"]/*[text()="Create model"]')
        createModelButton.text
        createModelButton.click()
        time.sleep(15)
        # Print success state
        print(
            f"Successfully created model {modelname} with hyperparams {hyperparams}")
    else:
        print(
            f"Prepared model with name {modelname}, but did not yet create it")


# Creates new model as clone from other model
def cloneModel(clonefrom, modelname, description, track, hyperparams,
               maxtime, rewardfunction=None, rfchanges=None, testmode=False):

    browser.get(
        "https://console.aws.amazon.com/deepracer/home?region=us-east-1#model/%s" % clonefrom)
    browser.refresh()
    time.sleep(3)

    # Click on clone button
    time.sleep(0.5)
    cloneButton = browser.find_element_by_xpath(
        '//*[@id="PLCHLDR_model_detail_clone_button"]')
    cloneButton.click()

    # Create model as clone
    time.sleep(0.5)
    createModel(modelname=modelname,
                description=description,
                track=track,
                hyperparams=hyperparams,
                maxtime=maxtime,
                rewardfunction=rewardfunction,
                rfchanges=rfchanges,
                testmode=testmode)


# Counts the number of models that are currently training
def count_models_training():

    browser.get(
        "https://console.aws.amazon.com/deepracer/home?region=us-east-1#models")
    browser.refresh()
    time.sleep(3)

    # Count number of models that are being created
    count_created = len(browser.find_elements_by_xpath(
        '//span/*[text()="Created"]'))
    # Count number of models that are training
    count_training = len(browser.find_elements_by_xpath(
        '//span/*[text()="Training..."]'))
    # Count number of models that are being stopped
    count_stopping = len(browser.find_elements_by_xpath(
        '//span/*[text()="Stopping..."]'))

    return count_created + count_training + count_stopping


# Clones model and performs multiple experiments with hyperparameters
def clone_hyperparams_experiment(clone_from_model, hyperparams_experiment, training_slots=2,
                                 number_of_experiments=2, start_naming_with="a", track="reInvent2019",
                                 maxtime_per_training=180):

    # Calculate approximate number of hours that this function will run
    total_hours = number_of_experiments * \
        (maxtime_per_training/60) / training_slots
    print(
        f"Starting {number_of_experiments} experiments. This will take approx {total_hours} hours.")

    # Create all combinations of hyperparameters. Result is list of dictionaries
    keys, values = zip(*hyperparams_experiment.items())
    hp_exp_all = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # Generate random indexes
    hp_exp_all_indexes = random.sample(
        range(len(hp_exp_all)), number_of_experiments)

    # Select only the dictionaries that were randomly selected
    hp_exp = [hp_exp_all[i] for i in hp_exp_all_indexes]

    # Transform e.g. "a" to integer 97
    start_naming_with_int = ord(start_naming_with)

    for hp_exp_i in hp_exp:

        # Wait until training slot is free
        slot_free = False
        while slot_free == False:
            if count_models_training() < training_slots:
                slot_free = True
            else:
                # wait 5 minutes before checking for free slot again
                time.sleep(5*60)

        model_creation_successful = False
        while model_creation_successful == False:
            try:
                # Start training of model with hyperparams hp_exp_i
                cloneModel(clonefrom=clone_from_model,
                           modelname=clone_from_model+"-clone-" +
                           chr(start_naming_with_int),
                           description=str(hp_exp_i)[1:-1],
                           track=track,
                           hyperparams=hp_exp_i,
                           maxtime=maxtime_per_training,
                           rfchanges=None,
                           testmode=False)
            except:
                # If model creation failed, print message and wait for 5 minutes
                current_version = chr(start_naming_with_int)
                now = datetime.datetime.now()
                now_h = now.hour
                now_m = now.minute
                print(
                    f"{now_h}:{now_m} Model creation of {current_version} failed. Trying again in 5 minutes")
                # Try logging in to AWS again, in case user was logged out
                try:
                    awsLogin()
                except:
                    pass
                time.sleep(5*60)
            else:
                time.sleep(5)
                # If not automatically transferred to new model url, print error message
                modelname_check = clone_from_model + \
                    "-clone-" + chr(start_naming_with_int)
                if browser.current_url != ("https://console.aws.amazon.com/deepracer/home?region=us-east-1#model/%s" % modelname_check):
                    print(f"Model creation of {modelname_check} may have failed.", end=' ')
                    print("Check in the console if model was created and create it manually if needed")

                # If model creation was successful, escape the while loop
                model_creation_successful = True
                # Increasing naming number
                start_naming_with_int += 1


def submit_to_spain(modelname):

    browser.get(
        "https://us-east-1.console.aws.amazon.com/deepracer/home?region=us-east-1#races/arn%3Aaws%3Adeepracer%3A%3A318512209165%3Aleaderboard%2Fd4f4275f-16aa-4a98-a752-bd8e82e463fe")
    browser.refresh()
    time.sleep(5)
    print('==========get enter race home==========')
    # Sometimes, pressing the submit button will not trigger a submit
    # Therefore, just retry 5 times
    re_press_submit = 5
    while re_press_submit > 0:
        try:
            print('==========get Race again==========0')
            submitToRaceButton = browser.find_element_by_xpath(
            "//span[contains(text(),'Race again')]")
            submitToRaceButton.click()
            print('==========get Race again==========')

            dropdown = browser.find_element_by_xpath(
            "//div[@class='']/button[@type='button']")
            dropdown.click()
            print('==========Choose model==========')
            choose_model = browser.find_element_by_xpath(
            "//div[@title='%s']" %modelname)
            choose_model.click()
            print('==========and submit it==========')
            
            enter_race = browser.find_element_by_xpath("//button[@type='submit']/span[contains(text(),'Enter race')]")
            enter_race.click()
            print('==========Enter race==========')
            re_press_submit -= 1
            time.sleep(2)
        except:
            print(error + 'Model Submission Failed')
            break
            # If click failed, means that submit was successful and we got re-routed to Event starting screen
            re_press_submit = 0

    time.sleep(15)

    print(f"{datetime.datetime.now()} Submitted model {modelname} to F1 Race")


def submit_to_spain_multiple(modelname, repeat_hours=9):

    # Calculate when to stop
    datetime_stop = datetime.datetime.now() + datetime.timedelta(hours=repeat_hours)

    # Count number of submits
    count_submits = 0
    count_fails = 0

    # Repeat loop until time is up
    while datetime.datetime.now() < datetime_stop:
        try:
            # Submit model to summit
            submit_to_spain(modelname=modelname)
            # Wait for 10 minutes before attempting submit again
            time.sleep(10*60)
        except:
            # If failed to submit, wait for 2 minutes and try again
            count_fails += 1
            time.sleep(2*60)
            # If failed 5 times, try to log back in
            if count_fails >= 10:
                awsLogin()
        else:
            # If there was no error, increase counter by 1
            count_submits += 1
            count_fails = 0

    # Print final submit count
    print(f"Submitted number of models to the race: {count_submits}")

# EXECUTE CODE

### Log in to AWS
- This cell should always be executed one time in the beginning
- Important: The directory of this notebook should have a AWS_credentials.txt file, which has 3 lines: AWS_id, username, and password. This will not work with an AWS root account

In [11]:
# Quits past browser instance
quitBrowser()

# Creates new browser instance
browser = newBrowser()

# Log in to AWS using AWS_credentials.txt
awsLogin()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_8572\1375102687.py:52: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  usernameInput = browser.find_elements_by_css_selector('form input')[1]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_8572\1375102687.py:53: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  passwordInput = browser.find_elements_by_css_selector('form input')[2]


Successfully logged in to AWS account number 318512209165 with username 18068840660@163.com


### Submit model to race multiple times

#### F1 Event (Spain track)

In [16]:
# Submit the model to the summit race for multiple hours
submit_to_spain_multiple(modelname="k1999-optimization-cw-3", repeat_hours=12)

==========get enter race home==========
==========get Race again==========0


C:\Users\Administrator\AppData\Local\Temp\ipykernel_8572\632674947.py:381: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  submitToRaceButton = browser.find_element_by_xpath(


==========get enter race home==========
==========get Race again==========0
==========get enter race home==========
==========get Race again==========0


KeyboardInterrupt: 

### Clone model and run experiments
This cell runs experiments on hyperparameters. Be aware, that this could take several hours to execute. Best to let it run over night.
1. Define hyperparameters that should be tested.
2. clone_from_model: Name of model, which should be cloned
3. training_slots: Maximum number of parallel training sessions that AWS allows (usually 4, but only 2 in May 2020)
4. number_of_experiments: As many combinations of hyperparameters are possible, only a hand full should be randomly selected and tested
5. start_naming_with: Defines with which character the clones should start being named. Be careful to start with a character that does not exist yet, otherwise AWS will give an error when trying to create the model
6. track: Track name (e.g. "reInvent2019" or "Spain")
7. maxtime_per_training: Define the number of minutes that each clone should be trained

In [ ]:
# All available hyperparameters
# "batchsize": [32,64,128,256,512],
# "epochs": [3 to 10],
# "learningrate": [0.00000001 to 0.001],
# "entropy": [0 to 1],
# "discount": [0 to 1],
# "episodesUpdate": [5 to 100]

experiment = {"batchsize":[64,256],
              "epochs":[5,10],
              "learningrate":[0.0001,0.0002,0.0003],
              "entropy":[0.005,0.01],
              "discount":[0.9993],
              "episodesUpdate":[20]}

clone_hyperparams_experiment(clone_from_model="Cup2019-v14-clone-e",
                             hyperparams_experiment=experiment,
                             training_slots=2,
                             number_of_experiments=24,
                             start_naming_with="a",
                             track="reInvent2019",
                             maxtime_per_training=180)